In [1]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas  as pd
import seaborn as sns
from pylab import rcParams

# Reading the CSV file (Excluding unnecessary)
df = pd.read_csv('https://raw.githubusercontent.com/ashishsahu1/ML-ProjectYard/main/Weather%20Forecasting/Dataset/Weather.csv',index_col=0) 

In [2]:
df.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


In [3]:
df1 = pd.melt(df, id_vars='YEAR', value_vars=df.columns[1:]) ## This will melt the data
df1.head()

,YEAR,variable,value
0,1901,JAN,17.99
1,1902,JAN,19.00
2,1903,JAN,18.32
3,1904,JAN,17.77
4,1905,JAN,17.40


In [4]:
from datetime import datetime
df1['Date'] = df1['variable'] + ' ' + df1['YEAR'].astype(str)  
df1.loc[:,'Date'] = df1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y')) ## Converting String to datetime object
df1.head()

,YEAR,variable,value,Date
0,1901,JAN,17.99,1901-01-01
1,1902,JAN,19.00,1902-01-01
2,1903,JAN,18.32,1903-01-01
3,1904,JAN,17.77,1904-01-01
4,1905,JAN,17.40,1905-01-01


In [62]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 

df2 = df1[['Year', 'Month', 'Temprature']].copy()
df2 = pd.get_dummies(df2)
y = df2[['Temprature']]
x = df2.drop(columns='Temprature')

dtr = DecisionTreeRegressor()
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.75)
dtr.fit(train_x, train_y)
pred = dtr.predict(test_x)
r2_score(test_y, pred)

0.9654043286090233

In [105]:
param_grid = {
              "min_samples_split": [3,4, 5, 7],
              "max_depth": [ 14, 15, 13],
              "min_samples_leaf": [2, 4, 3, 5],
              "max_leaf_nodes": [800, 650, 700, 750],
              }


grid_cv_dtm = GridSearchCV(dtr, param_grid, cv=5)

grid_cv_dtm.fit(train_x,train_y)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [14, 15, 13],
                         'max_leaf_nodes': [800, 650, 700, 750],
                         'min_samples_leaf': [2, 4, 3, 5],
                         'min_samples_split': [3, 4, 5, 7]})

In [106]:
print("R-Squared::{}".format(grid_cv_dtm.best_score_))
print("Best Hyperparameters::\n{}".format(grid_cv_dtm.best_params_))

R-Squared::0.9727400870199716
Best Hyperparameters::
{'max_depth': 13, 'max_leaf_nodes': 800, 'min_samples_leaf': 5, 'min_samples_split': 3}


In [107]:
Y_pred = grid_cv_dtm.predict(test_x)
print(Y_pred)
print('Decision Tree Regressor RMSE',np.sqrt(mean_squared_error(test_y,Y_pred)))
print('Decision Tree Regressor MAE',mean_absolute_error(test_y,Y_pred))
print('Decision Tree Regressor MSE',mean_squared_error(test_y,Y_pred))

[18.86333333 27.58       27.13       ... 26.876      21.488
 27.102     ]
Decision Tree Regressor RMSE 0.5791139673261005
Decision Tree Regressor MAE 0.42772768205731193
Decision Tree Regressor MSE 0.3353729871521758


In [108]:
r2_score(test_y, Y_pred)

0.973302537193806

In [63]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,mean_absolute_error

In [64]:
reg = RandomForestRegressor()
reg.fit(train_x,train_y)

<ipython-input-64-eae6b46f2258>:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor()

In [65]:
Y_pred = reg.predict(test_x)
print(Y_pred)
r2_score(test_y, Y_pred)

[19.1322 27.6607 27.0746 ... 26.9324 21.8752 27.1649]


0.9727955322173371

In [66]:
print('Random Forest Regression for Test RMSE',np.sqrt(mean_squared_error(test_y,Y_pred)))
print('Random Forest Regression for Test MSE',mean_squared_error(test_y,Y_pred))
print('Random Forest Regression for Test MAE',mean_absolute_error(test_y,Y_pred))

Random Forest Regression for Test RMSE 0.5845870109742521
Random Forest Regression for Test MSE 0.3417419733998104
Random Forest Regression for Test MAE 0.4323651471984803


In [102]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [34]:
random_grid = {
          'max_depth': [50,60,70],
          'max_features': ['auto', 'sqrt'],
          'min_samples_leaf': [1, 2, 3],
          'min_samples_split': [2, 3, 4, 5],
          'n_estimators': [500, 400, 600, 700 ]
         }

In [39]:
rf = RandomForestRegressor()

In [40]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring = 'neg_mean_squared_error',
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [41]:
rf_random.fit(train_x,train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.7min finished
C:\Users\Dishita Neve\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [50, 60, 70],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 3, 4, 5],
                                        'n_estimators': [500, 400, 600, 700]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [42]:
rf_random.best_params_

{'n_estimators': 700,
 'min_samples_split': 3,
 'min_samples_leaf': 3,
 'max_features': 'sqrt',
 'max_depth': 60}

In [43]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=60, max_features='sqrt', min_samples_leaf=3,
                      min_samples_split=3, n_estimators=700)

In [97]:
reg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=65,
                      max_features=7, max_leaf_nodes=None,
                      min_impurity_decrease=0.00, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=3,
                      min_weight_fraction_leaf=0.00, n_estimators=700,
                      n_jobs=-1,oob_score=False, random_state=0,
                      verbose=0,warm_start=False)

In [98]:
reg.fit(train_x,train_y)

<ipython-input-98-ac6777d7b2d7>:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(max_depth=65, max_features=7, min_samples_leaf=3,
                      min_samples_split=3, n_estimators=700, n_jobs=-1,
                      random_state=0)

In [99]:
Y_pred = reg.predict(test_x)
print(Y_pred)
r2_score(test_y, Y_pred)

[18.8404979  27.52581427 27.22018004 ... 26.84521201 21.4895624
 27.19242003]


0.9752239951950142

In [100]:
print('Random Forest Regression for Test RMSE',np.sqrt(mean_squared_error(test_y,Y_pred)))
print('Random Forest Regression for Test MSE',mean_squared_error(test_y,Y_pred))
print('Random Forest Regression for Test MAE',mean_absolute_error(test_y,Y_pred))

Random Forest Regression for Test RMSE 0.5578849976923617
Random Forest Regression for Test MSE 0.3112356706502064
Random Forest Regression for Test MAE 0.4094741511417105


In [79]:
import xgboost
from xgboost import XGBRegressor

In [58]:
xgr = XGBRegressor()
xgr.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
Y_pred = xgr.predict(test_x)
print(Y_pred)

[18.695599 26.45061  29.342468 ... 26.901333 26.815075 24.548018]


C:\Users\Dishita Neve\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



In [73]:
r2_score(test_y, Y_pred)

0.9708850245789332

In [72]:
print('XG Boost for Test RMSE',np.sqrt(mean_squared_error(test_y,Y_pred)))
print('XG Boost for Test MSE',mean_squared_error(test_y,Y_pred))
print('XG Boost for Test MAE',mean_absolute_error(test_y,Y_pred))

XG Boost for Test RMSE 0.6047658486273623
XG Boost for Test MSE 0.3657417316659737
XG Boost for Test MAE 0.4519476304148154
